In [1]:
# pip install xgboost
# pip install lightgbm
# pip install catboost
import gc
import os
import pandas as pd
import catboost as cbt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
seed = 2022
df_feature = pd.read_pickle('data/feature.pkl')
for f in df_feature.select_dtypes('object').columns:
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

df_train = df_feature[df_feature.isDefault.notna()].copy()
df_test = df_feature[df_feature.isDefault.isna()].copy()
df_train.shape, df_test.shape

((800000, 341), (200000, 341))

In [4]:
ycol = 'isDefault'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], df_train.columns))

# Logistic Regression

In [5]:
df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in list(df_train.columns[df_train.isnull().sum() > 0]):
    mean_val = df_train[column].mean()
    df_train[column].fillna(mean_val, inplace=True)

df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
for column in list(df_test.columns[df_test.isnull().sum() > 0]):
    mean_val = df_test[column].mean()
    df_test[column].fillna(mean_val, inplace=True)

In [6]:
# dense_features = ['annualIncome', 'delinquency_2years', 'dti', 'employmentLength', 'ficoRangeHigh',
#                   'ficoRangeLow', 'installment', 'interestRate', 'loanAmnt', 'openAcc', 'pubRec',
#                   'pubRecBankruptcies', 'revolBal', 'revolUtil', 'subGrade', 'term', 'totalAcc',
#                   'homeOwnership', 'purpose', 'policyCode']
# df_feature[dense_features] = (df_feature[dense_features] - df_feature[dense_features].mean()) / df_feature[dense_features].std()

df_feature = (df_feature - df_feature.mean()) / df_feature.std()  # 

In [7]:
df_oof = df_train[['id', ycol]].copy()  # df_train
df_oof['prob'] = 0   # 800000, 2

# Test set
prediction = df_test[['id']]
prediction['prob'] = 0   # 200000, 2

model = LogisticRegression(solver='sag')  

# CV5
# Train/Validation/Test set
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print("\ntraining Fold:{}\n".format(fold_id+1))
    lr_model = model.fit(X_train, Y_train)
    
    pred_val = lr_model.predict_proba(X_val)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val   # validation prob

    pred_test = lr_model.predict_proba(df_test[feature_names])[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits
    del lr_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('在所有训练集上的auc:', auc)

os.makedirs('sub', exist_ok=True)
sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/lr_jiedai_{}.csv'.format(auc), index=False)

os.makedirs('prob', exist_ok=True)
prediction.to_csv('prob/lr_test.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/lr_train.csv', index=False)

# auc: 0.6179224043661626

/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/324089590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] = 0   # 200000, 2



training Fold:1



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/324089590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits



training Fold:2



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/324089590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits


...


training Fold:4



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/324089590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits



training Fold:5



/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/324089590.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits


在所有训练集上的auc: 0.6179244934618311


# Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
df_oof = df_train[['id', ycol]].copy()  # df_train
df_oof['prob'] = 0   # 800000, 2

# 测试集
prediction = df_test[['id']]
prediction['prob'] = 0   # 200000, 2

model = RandomForestClassifier(n_estimators=1000, criterion="entropy", max_depth=6)

# CV5
# 训练集、验证集、测试集
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names], df_train[ycol])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]
    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print("\ntraining Fold:{}\n".format(fold_id+1))
    lr_model = model.fit(X_train, Y_train)
    
    pred_val = lr_model.predict_proba(X_val)[:, 1]
    df_oof.loc[val_idx, 'prob'] = pred_val   # 验证集概率

    pred_test = lr_model.predict_proba(df_test[feature_names])[:, 1]
    prediction['prob'] += pred_test / kfold.n_splits
    del lr_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()

auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
print('在所有训练集上的auc:', auc)

os.makedirs('sub', exist_ok=True)
sub = prediction[['id', 'prob']].copy()
sub.columns = ['id', 'isDefault']
sub.to_csv('sub/rf_jiedai_{}.csv'.format(auc), index=False)

os.makedirs('prob', exist_ok=True)
prediction.to_csv('prob/rf_test.csv', index=False)
df_oof[['id', 'prob', ycol]].to_csv('prob/rf_train.csv', index=False)

# auc: 0.7093926062158106

/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/4067556301.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] = 0   # 200000, 2



training Fold:1



/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/4067556301.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits



training Fold:2



/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/4067556301.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits


...


training Fold:4



/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/4067556301.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits



training Fold:5



/var/folders/tb/f0rvdbv503x9jd71nm38z_2h0000gn/T/ipykernel_26270/4067556301.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction['prob'] += pred_test / kfold.n_splits


在所有训练集上的auc: 0.7094246490213467


# XGBoost

In [10]:
# xgboost
# pip install xgboost  
# pip install sklearn
# pip install tqdm
# conda install xgboost
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gc
import os
from tqdm import tqdm
import numpy as np

warnings.simplefilter('ignore')


if __name__ == '__main__':   
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')
    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    

    df_train = df_feature[df_feature.isDefault.notna()].copy()   
    df_test = df_feature[df_feature.isDefault.isna()].copy()   

    #df_train = clean_dataset(df_train)
    #df_test = clean_dataset(df_test)
    #df_train = df_train.dropna(axis=0, how='any')   # drop rows containing nan
    df_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_train.fillna(df_train.mean(), inplace=True)   # fillna with medians
    df_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_test.fillna(df_test.mean(), inplace=True)
    
    print(df_train.shape, df_test.shape)   # (800000, 46+xx)

    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))

    
    model = xgb.XGBClassifier(max_depth=6,          
                              learning_rate=0.05,   
                              n_estimators=10000,  
                              subsample=0.8,       
                              reg_alpha=10,         # l1/l0
                              reg_lambda=12,        # l2
                              # tree_method='gpu_hist',
                              random_state=seed)

    # id  prob   # predictions on train set
    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0

    # id prob  # predictions on test set
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)  # cv5
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]
        # print(X_train.shape)   #  800000 / 5 * 4  (640000, 337)  46
        # print(Y_train.shape)   # (640000, 1)   #

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]
        # print(X_val.shape)   # 160000, 337
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        # E extreme xgboost  gbdt
        xgb_model = model.fit(X_train,
                              Y_train,
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=100,
                              eval_metric='auc',
                              early_stopping_rounds=50) 

        pred_val = xgb_model.predict_proba(X_val)[:, 1]   # [0.2, 0.8]
        df_oof.loc[val_idx, 'prob'] = pred_val
      

        #
        pred_test = xgb_model.predict_proba(df_test[feature_names])[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': xgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del xgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby([
        'column'
    ])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    print(df_importance)
    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)
    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/jiedai_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)

    prediction.to_csv('prob/sub_xgb.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_xgb.csv', index=False)

    prediction.to_csv('prob/sub_xgb_{}.csv'.format(auc), index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_xgb_{}.csv'.format(auc), index=False)

    # auc: 0.7430355546194591

(800000, 341) (200000, 341)

Fold_1 Training ================================

[0]	validation_0-auc:0.70799	validation_1-auc:0.70577
[100]	validation_0-auc:0.74107	validation_1-auc:0.73312
[200]	validation_0-auc:0.75098	validation_1-auc:0.73819
[300]	validation_0-auc:0.75761	validation_1-auc:0.74006
[400]	validation_0-auc:0.76330	validation_1-auc:0.74101
[500]	validation_0-auc:0.76861	validation_1-auc:0.74159
[600]	validation_0-auc:0.77342	validation_1-auc:0.74189
[700]	validation_0-auc:0.77813	validation_1-auc:0.74204
[800]	validation_0-auc:0.78273	validation_1-auc:0.74223
[900]	validation_0-auc:0.78720	validation_1-auc:0.74238
[1000]	validation_0-auc:0.79154	validation_1-auc:0.74251
[1033]	validation_0-auc:0.79293	validation_1-auc:0.74245

Fold_2 Training ================================

[0]	validation_0-auc:0.70821	validation_1-auc:0.70458
[100]	validation_0-auc:0.74134	validation_1-auc:0.73200
[200]	validation_0-auc:0.75120	validation_1-auc:0.73669
[300]	validation_0-auc:0.75798	v

# LightGBM

In [11]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os


if __name__ == '__main__':

    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')

    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    df_train = df_feature[df_feature.isDefault.notna()].copy()
    df_test = df_feature[df_feature.isDefault.isna()].copy()
    print(df_train.shape, df_test.shape)
    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))
    # pycharm
    model = lgb.LGBMClassifier(objective='binary',
                               boosting_type='gbdt',
                               num_leaves=32,
                               max_depth=6,
                               learning_rate=0.05,
                               n_estimators=10000,
                               subsample=0.8,
                               feature_fraction=0.6,
                               reg_alpha=10,
                               reg_lambda=12,
                               random_state=seed,
                               is_unbalance=True,
                               metric='auc')

    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        lgb_model = model.fit(X_train,
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=100,
                              early_stopping_rounds=50)

        pred_val = lgb_model.predict_proba(
            X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
        df_oof.loc[val_idx, 'prob'] = pred_val

        pred_test = lgb_model.predict_proba(
            df_test[feature_names], num_iteration=lgb_model.best_iteration_)[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': lgb_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)
    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/lgb_test_result_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)
    prediction.to_csv('prob/sub_lgb.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_lgb.csv', index=False)
    # auc: 7421851952531208

(800000, 341) (200000, 341)

Fold_1 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[100]	train's auc: 0.739558	valid's auc: 0.733217
[200]	train's auc: 0.747537	valid's auc: 0.73755
[300]	train's auc: 0.752945	valid's auc: 0.739387
[400]	train's auc: 0.757407	valid's auc: 0.740425
[500]	train's auc: 0.761388	valid's auc: 0.740865
[600]	train's auc: 0.765017	valid's auc: 0.741124
[700]	train's auc: 0.768447	valid's auc: 0.741419
[800]	train's auc: 0.771822	valid's auc: 0.741543
[900]	train's auc: 0.775174	valid's auc: 0.741648
[1000]	train's auc: 0.778358	valid's auc: 0.741736
[1100]	train's auc: 0.781423	valid's auc: 0.741745

Fold_2 Training ================================

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[100]	train's auc: 0.739935	valid's auc: 0.732507
[200]	train's auc: 0.

# CatBoost

In [12]:
import gc
import os
import pandas as pd
import catboost as cbt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold


if __name__ == '__main__':
    #pd.set_option('max_columns', None)
    #pd.set_option('max_rows', None)

    seed = 2022
    df_feature = pd.read_pickle('data/feature.pkl')
    for f in df_feature.select_dtypes('object').columns:
        lbl = LabelEncoder()
        df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

    df_train = df_feature[df_feature.isDefault.notna()].copy()
    df_test = df_feature[df_feature.isDefault.isna()].copy()
    print(df_train.shape, df_test.shape)

    ycol = 'isDefault'
    feature_names = list(
        filter(lambda x: x not in [ycol, 'id'], df_train.columns))

    model = cbt.CatBoostClassifier(eval_metric='AUC',
                                   max_depth=6,
                                   learning_rate=0.05,
                                   n_estimators=10000,
                                   reg_lambda=12,
                                   # task_type='GPU',
                                   random_state=seed)

    df_oof = df_train[['id', ycol]].copy()
    df_oof['prob'] = 0
    prediction = df_test[['id']]
    prediction['prob'] = 0
    df_importance_list = []

    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for fold_id, (trn_idx, val_idx) in enumerate(
            kfold.split(df_train[feature_names], df_train[ycol])):
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][ycol]

        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][ycol]

        print('\nFold_{} Training ================================\n'.format(
            fold_id + 1))

        cbt_model = model.fit(X_train,
                              Y_train,
                              eval_set=[(X_val, Y_val)],
                              verbose=100,
                              early_stopping_rounds=50)

        pred_val = cbt_model.predict_proba(
            X_val)[:, 1]
        df_oof.loc[val_idx, 'prob'] = pred_val

        pred_test = cbt_model.predict_proba(
            df_test[feature_names])[:, 1]
        prediction['prob'] += pred_test / kfold.n_splits

        df_importance = pd.DataFrame({
            'column': feature_names,
            'importance': cbt_model.feature_importances_,
        })
        df_importance_list.append(df_importance)

        del cbt_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()

    df_importance = pd.concat(df_importance_list)
    df_importance = df_importance.groupby([
        'column'
    ])['importance'].agg('mean').sort_values(ascending=False).reset_index()
    print(df_importance)

    auc = roc_auc_score(df_oof[ycol], df_oof['prob'])
    print('auc:', auc)

    os.makedirs('sub', exist_ok=True)

    sub = prediction[['id', 'prob']].copy()
    sub.columns = ['id', 'isDefault']
    sub.to_csv('sub/cat_test_result_{}.csv'.format(auc), index=False)
    os.makedirs('prob', exist_ok=True)

    prediction.to_csv('prob/sub_cbt.csv', index=False)
    df_oof[['id', 'prob', ycol]].to_csv('prob/oof_cbt.csv', index=False)
    # AUC: 0.7433677897801529

(800000, 341) (200000, 341)

Fold_1 Training ================================

0:	test: 0.6971665	best: 0.6971665 (0)	total: 249ms	remaining: 41m 33s
100:	test: 0.7296939	best: 0.7296939 (100)	total: 11.6s	remaining: 18m 58s
200:	test: 0.7344384	best: 0.7344384 (200)	total: 22s	remaining: 17m 54s
300:	test: 0.7368384	best: 0.7368384 (300)	total: 32.4s	remaining: 17m 24s
400:	test: 0.7384861	best: 0.7384861 (400)	total: 42.6s	remaining: 16m 59s
500:	test: 0.7394207	best: 0.7394207 (500)	total: 52.8s	remaining: 16m 40s
600:	test: 0.7400946	best: 0.7400946 (600)	total: 1m 2s	remaining: 16m 23s
700:	test: 0.7405985	best: 0.7405985 (700)	total: 1m 12s	remaining: 16m 2s
800:	test: 0.7408938	best: 0.7409046 (797)	total: 1m 22s	remaining: 15m 46s
900:	test: 0.7411860	best: 0.7411860 (900)	total: 1m 32s	remaining: 15m 29s
1000:	test: 0.7414844	best: 0.7414875 (997)	total: 1m 41s	remaining: 15m 13s
1100:	test: 0.7416960	best: 0.7416960 (1100)	total: 1m 51s	remaining: 15m
1200:	test: 0.7418350	be

In [13]:
import pandas as pd
from sklearn.metrics import roc_auc_score


if __name__ == '__main__':
    # 官方给的测试集的概率预测
    lgb_sub = pd.read_csv('prob/sub_lgb.csv')
    xgb_sub = pd.read_csv('prob/sub_xgb.csv')
    cbt_sub = pd.read_csv('prob/sub_cbt.csv')
#     print(lgb_sub.shape)  # 200000, 2   # (200000, 2)

    lgb_oof = pd.read_csv('prob/oof_lgb.csv')
    xgb_oof = pd.read_csv('prob/oof_xgb.csv')
    cbt_oof = pd.read_csv('prob/oof_cbt.csv')
#     print(lgb_oof.shape)   # 800000, 3     id prob label
        
    
    # 三个模型测试集的预测结果合并成一个表
    sub = lgb_sub.copy()
    sub.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
    sub['prob_xgb'] = xgb_sub['prob'].values.tolist()
    sub['prob_cbt'] = cbt_sub['prob'].values.tolist()
    print(sub.shape)   # (200000, 4)   # id, prob_lgb, prob_xgb, prob_cbt
    
    # 三个模型训练集的预测结果合并成一个表
    oof = lgb_oof.copy()
    oof.rename({'prob': 'prob_lgb'}, axis=1, inplace=True)
    oof['prob_xgb'] = xgb_oof['prob'].values.tolist()
    oof['prob_cbt'] = cbt_oof['prob'].values.tolist()
    # print(oof.head())
    print(oof.shape)  # id, isDefault, prob_lgb, prob_xgb, prob_cbt

    # 计算auc
    auc1 = roc_auc_score(oof['isDefault'], oof['prob_lgb'])
    auc2 = roc_auc_score(oof['isDefault'], oof['prob_xgb'])
    auc3 = roc_auc_score(oof['isDefault'], oof['prob_cbt'])
    print('lgb的auc:', auc1)   # 0.7427987878983419
    print('xgb的auc:', auc2)   # 0.7434926337868603
    print('cbt的auc:', auc3)   # 0.4994859248920798

    
    oof['three'] = (oof['prob_lgb'] + oof['prob_xgb'] + oof['prob_cbt'])/3
    avg_auc = roc_auc_score(oof['isDefault'], oof['three'])
    print('三个均值的auc:', avg_auc)   
        
    
    w_lgb = 0.5
    w_xgb = 0.5
    w_cbt = 0.5
    # x y x = 根号x * 根号y * 根号 z
    oof['prob'] = oof['prob_lgb'] ** w_lgb * oof['prob_xgb'] ** w_xgb * oof['prob_cbt'] ** w_cbt
    auc = roc_auc_score(oof['isDefault'], oof['prob'])
    print('根号相乘的auc:', auc)
    
    oof['both'] = (oof['prob_xgb'] + oof['prob_cbt'])/2
    
    both_auc = roc_auc_score(oof['isDefault'], oof['both'])
    print('xgb and cbt:', both_auc)
    
    sub['prob'] = sub['prob_lgb'] ** w_lgb * sub['prob_xgb'] ** w_xgb * sub['prob_cbt'] ** w_cbt
    # sub['prob'] = (sub['prob_lgb'] + sub['prob_xgb']) / 2
    sub.rename({'prob': 'isDefault'}, axis=1, inplace=True)
    sub[['id', 'isDefault']].to_csv('sub/predict.csv', index=False)

(200000, 4)
(800000, 5)
lgb的auc: 0.7421851952531208
xgb的auc: 0.7430355546194591
cbt的auc: 0.7428531664426238
三个均值的auc: 0.7436647734570626
根号相乘的auc: 0.7438512708269246
xgb and cbt: 0.743844773578163


In [14]:
# df_test.head()